In [1]:
!pip install -U deep_translator
!pip install pandas
!pip install transformers

     |████████████████████████████████| 128 kB 39.7 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
     |████████████████████████████████| 4.0 MB 17.5 MB/s 
     |████████████████████████████████| 596 kB 45.9 MB/s 
     |████████████████████████████████| 6.6 MB 37.5 MB/s 
     |████████████████████████████████| 84 kB 1.8 MB/s 
     |████████████████████████████████| 880 kB 36.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=7248345a5cdae31a1aa07da788f6e4d3d5b3f8fb35cfec4b77e4aa4cd9fb81ac
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import pipeline
import pandas as pd
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
from scipy.special import softmax
import numpy as np
import urllib.request
import csv

In [4]:
#TRANSLATOR
from deep_translator import GoogleTranslator
translated = GoogleTranslator(source='auto', target='es')

In [5]:
#Create dataframe

import pandas as pd

data = {"Fact": ["The opposite of Microsoft Office is Macrohard Onfire"], "FactES":["El opuesto a Microsoft Office es Macrohard Onfire"], "Holiday":["General"], "Topic": ["Tech"]}
post = pd.DataFrame(data)
post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Process Dataframes/Facts.xlsx")

In [6]:
#Read the wrapper dataframe

data = pd.read_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Wrappers/Facts.xlsx", index_col=0)

In [7]:
#Sentimental Analysis

#DISTILBERT
distilbert = pipeline("text-classification",model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

#TWITTER ROBERTA
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

#MODEL FOR TOPIC CLASSIFICATION
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

candidate_labels = ['technology', 'science', 'health', 'economic', "maths", "politics", "chemistry", "animals", "law"]

Downloading:   0%|          | 0.00/768 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/291 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [8]:
#Extract the facts and apply the models

for i in range(len(data)) :
  #print(data.loc[i, "Fact"])
  fact = data.loc[i, "Fact"]
  #SENTIMENTAL ANALYSIS 1
  #TWITTER ROBERTA  
  encoded_input = tokenizer(fact, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)

  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  l = labels[ranking[0]]
  s = scores[ranking[0]]
  #If the higher score is negative (always the higher score comes in the 0 position)
  if l == "negative":
    continue
  
  #SENTIMENTAL ANALYSIS 1
  #DISTILBERT
  flag = False
  prediction = distilbert(fact, )
  #if the classification is mainly sadness, anger or fear
  for e in prediction:
    for x in e:
      if x["label"] == "sadness":
        if x["score"] > 0.6:
          flag = True
      elif x["label"] == "anger":
        if x["score"] > 0.6:
          flag = True
      elif x["label"] == "fear":
        if x["score"] > 0.6:
          flag = True
  if flag:
    continue

  #TOPIC CLASSIFICATION
  topicList = classifier(fact, candidate_labels)
  if (topicList["labels"][0] == "technology") or (topicList["labels"][0] == "science") or (topicList["labels"][0] == "maths"):
    topic = "Tech"
  elif (topicList["labels"][0] == "health") or (topicList["labels"][0] == "chemistry") or (topicList["labels"][0] == "animals"):
    topic = "Health"
  elif (topicList["labels"][0] == "economic") or (topicList["labels"][0] == "politics") or (topicList["labels"][0] == "law"):
    topic = "Law"


  trans = translated.translate(fact)
  prueba = {"Fact":data.loc[i, "Fact"], "FactES":trans, "Holiday":data.loc[i, "Holiday"], "Topic": topic}
  post = post.append(prueba, ignore_index=True)

In [9]:
post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Project/Process Dataframes/Facts.xlsx")